# Description

Pré-traitement des fichiers ".parquet" correspondant à des séries temporelles

Dataset de sortie :  
- une ligne par heure et par patient entre son entrée et sa sortie
- une colonne par feature
-  Nan si pas de valeur pour cette heure et ce patient
- optionnel : 2e dataset censuré à 7 jours. Pour les patients sorties avant : Comblement avec Nan jusqu'à J7

1. import des séries  
2. Retrait des valeurs antérieurs à l'admission (offset de -1h en cas de bug à l'admission)  
3. regroupement des données par heure 
    - pour les pressions artérielles : regroupement invasif et non invasif avec priorité sur l'invasif
4. Ajout de NaN sur les heures manquantes (entre 1ère heure et dernière heure)


# 1.Import

## 1.1 Librairies

In [ ]:
import polars as pl
import numpy as np
import os
import json
from tqdm import tqdm

## 1.2 Constantes

In [ ]:
with open('../params.json', 'r') as file :
    params = json.load(file)

DATASET, VERSION = params['dataset'], params['version']


In [ ]:
# Constantes
CENSUS_FILE = f'/data2/poette.m/dypo/{VERSION}/2.clean_data/{DATASET}/static/clean_static_encounters.parquet'
INPUT_FOLDER = f'/data2/poette.m/dypo/{VERSION}/1.raw_data/{DATASET}/dynamic_features/'
OUTPUT_FOLDER = f'/data2/poette.m/dypo/{VERSION}/2.clean_data/{DATASET}/temporal/'
with open('temporal_range.json', 'r') as f:
    temporal_range = json.load(f)

## 1.3 Fonctions

### 1.3.1 Nettoyage du dataset

In [ ]:
test_df = pl.read_parquet(INPUT_FOLDER + 'pam_invasive.parquet')

In [ ]:
def cleared_df(df, feature) :
    feature_range = feature.replace('_non_invasive', '').replace('_invasive', '')
    
    print(feature)
    if feature in ['pep', 'fio2'] : 
        lower_bound, upper_bound = temporal_range[feature_range]['range']
    else : 
        # A ajouter Loop par patient 
        mean = df['valueNumber'].mean()
        ds = df['valueNumber'].std()
        stat_lower_bound = mean - (3 * ds)
        stat_upper_bound = mean + 3*ds
        set_lower_bound, set_upper_bound = temporal_range[feature_range]['range']
        lower_bound = max(stat_lower_bound, set_lower_bound)
        upper_bound = min(stat_upper_bound, set_upper_bound)
    df_cleared = (df
        .select(
                ['encounterId', 'delta_inTime_hours', 'valueNumber', 'feature']
        )
        # Retrait des données antérieurs à l'admission et des valeurs hors range
        .filter(
            pl.col('delta_inTime_hours') >= -1,
            pl.col('valueNumber').is_between(lower_bound, upper_bound)
        )
        # Troncature de l'intervalle de la données par rapport à l'entrée             
        .with_columns(
            (pl.col("delta_inTime_hours").cast(pl.Int64)).alias('intervalle')
        )
        
        .group_by(
            'encounterId', 'intervalle'
        )
        .agg(
            pl.col("valueNumber").median().alias(feature)
        )
        .sort(
            'encounterId', 'intervalle'
        )
    )
    return df_cleared

In [ ]:
def cleared_df(df, feature):
    feature_range = feature.replace('_non_invasive', '').replace('_invasive', '')
    
    lower_bound, upper_bound = temporal_range[feature_range]['range']
    
    # Calculer la moyenne et l'écart type pour chaque encounterId
    stats_df = df.group_by('encounterId').agg([
        pl.col('valueNumber').mean().alias('mean'),
        pl.col('valueNumber').std().alias('std')
    ])
    
    # Joindre les statistiques avec le dataframe original
    df = df.join(stats_df, on='encounterId')
    
    # Calculer les bornes dynamiques pour chaque encounterId
    df = df.with_columns([
        (pl.col('mean') - 2 * pl.col('std')).alias('dynamic_lower_bound'),
        (pl.col('mean') + 2 * pl.col('std')).alias('dynamic_upper_bound')
    ])
    
    # Appliquer les filtres
    df_cleared = (df
        .select(
            ['encounterId', 'delta_inTime_hours', 'valueNumber', 'feature', 'dynamic_lower_bound', 'dynamic_upper_bound']
        )
        # Retrait des données antérieures à l'admission et des valeurs hors range
        .filter(
            (pl.col('delta_inTime_hours') >= -1) &
            (pl.col('valueNumber').is_between(lower_bound, upper_bound)) &
            (pl.col('valueNumber').is_between(pl.col('dynamic_lower_bound'), pl.col('dynamic_upper_bound')))
        )
        # Troncature de l'intervalle de la donnée par rapport à l'entrée
        .with_columns(
            (pl.col("delta_inTime_hours").cast(pl.Int64)).alias('intervalle')
        )
        .group_by(
            'encounterId', 'intervalle'
        )
        .agg(
            pl.col("valueNumber").median().alias(feature)
        )
        .sort(
            'encounterId', 'intervalle'
        )
    )
    
    return df_cleared


###  1.3.2 Traitements des valeurs de pression artérielle

In [ ]:
pressures_features = ['pam', 'pas', 'pad']
non_invasive_pressures = [pressure +'_non_invasive' for pressure in pressures_features]
invasive_pressures = [pressure +'_invasive' for pressure in pressures_features]

# Fonction pour traiter les datasets de pressions
def merged_pressure(invasive_df, non_invasive_df, feature_name):
    # Lire les datasets invasifs et non invasifs

    # Fusionner les datasets en priorisant les valeurs invasives
    merged_df = (
        invasive_df
        .join(
            non_invasive_df, on=["encounterId", "intervalle"], how="full"
        )
        .with_columns(
            # Priorité aux valeurs invasives, compléter avec non-invasives si nécessaire
            pl.coalesce([pl.col(f'{feature_name}_invasive'), pl.col(f'{feature_name}_non_invasive')]).alias(feature_name)
        )
        .with_columns(
           pl.col("encounterId").fill_null(pl.col("encounterId_right")),
           pl.col("intervalle").fill_null(pl.col("intervalle_right"))
        )
        .select(
            ['encounterId', 'intervalle', feature_name]
        )

    )
    return merged_df

### 1.3.3 Comblement des intervalles manquants par Null

In [ ]:
def fill_missing_intervalle(df, feature) :
    df_with_null = (
        df
        # Trouver l'intervalle maximum pour chaque patient
        .group_by("encounterId")
        
        .agg([
            pl.col("intervalle").max().alias("max_inter")
        ])

        # Étendre l'intervalle pour inclure toutes les heures entre 0 et max_hour
        .with_columns([
            pl.struct(
                ["encounterId", "max_inter"]
                ).map_elements(
                lambda row: list(range(0, row["max_inter"] + 1)),
                return_dtype=pl.List(pl.Int64)
                ).alias("all_hours")
        ])

        # Exploser les heures dans une nouvelle ligne
        .explode("all_hours")

        # Joindre avec les données existantes pour aligner les heures
        .join(
            df,
            left_on=["encounterId", "all_hours"],
            right_on=["encounterId", "intervalle"],
            how="left"
        )

        # Remplacer les valeurs manquantes par des NaN dans les colonnes de features

        .with_columns([
            pl.col(feature).fill_null(float('nan')) for feature in df.columns if feature not in ["encounterId", "intervalle"]
        ])
        
        # Renommer les colonnes pour uniformité
        .rename({"all_hours": "intervalle"})

        # Réorganiser les colonnes
        .select(["encounterId", "intervalle", feature])
    )
    return df_with_null

# Script

## Cleaning features

In [ ]:
DATASET

In [ ]:
VERSION

In [ ]:
encounters = pl.read_parquet(CENSUS_FILE)

In [ ]:
for filename in os.listdir(INPUT_FOLDER):

    if filename.endswith('.parquet'):


        feature = os.path.splitext(filename)[0]

        if feature not in (non_invasive_pressures + invasive_pressures) :
            
            raw_df = pl.read_parquet(os.path.join(INPUT_FOLDER, filename))
            cleared = cleared_df(raw_df, feature)
            with_missing_values = fill_missing_intervalle(cleared, feature)
            
            with_missing_values.write_parquet(os.path.join(OUTPUT_FOLDER, f'cleared_{filename}'))
            print(f'ok {feature}')
            
        
        elif feature in invasive_pressures :
            pressure_feature = feature.replace('_invasive', '')
            non_invasive_feature = f'{pressure_feature}_non_invasive'
            raw_invasive = pl.read_parquet(os.path.join(INPUT_FOLDER, filename))
            
            raw_non_invasive = pl.read_parquet(os.path.join(INPUT_FOLDER, filename.replace('invasive', 'non_invasive')))

            cleared_invasive = cleared_df(raw_invasive, feature)
            cleared_non_invasive = cleared_df(raw_non_invasive, non_invasive_feature)
            merged_df = merged_pressure(cleared_invasive, cleared_non_invasive, pressure_feature)
            with_missing_values = fill_missing_intervalle(merged_df, pressure_feature)

            with_missing_values.write_parquet(os.path.join(OUTPUT_FOLDER, f'cleared_{pressure_feature}.parquet'))
            print(f'ok {pressure_feature}')
            

## Merge in 1 week dataset

In [ ]:
def reset_first_value(df) :
    cleaned_dfs = []
    features_col = list(set(df.columns) - set(['encounterId', 'intervalle']))
    for encounterId, group in df.group_by("encounterId"):
        # Vérifie si au moins une variable n'est pas nulle
        group = (group
                    .fill_nan(None)
                    .with_columns(
                        pl.any_horizontal(features_col).is_not_null()
                        .alias('has_data')
                        )
                )
        # Trouver l'index de la première ligne où la variable has_data est True
        first_valid_index = group.select(pl.col("has_data")).to_pandas()["has_data"].idxmax()

        # Si aucun élément n'est valide, ignorer le patient
        if first_valid_index == -1:
            continue

        # Garder les lignes à partir de la première valide
        group = group[first_valid_index:]

        # Réinitialiser l'intervalle pour commencer à zéro
        group = group.with_columns(
            (pl.col("intervalle") - pl.lit(group["intervalle"][0])).alias("intervalle")
        )
        
        # Supprimer la colonne temporaire "has_data"
        group = group.drop("has_data")


        cleaned_dfs.append(group)

    # Fusionner tous les groupes nettoyés
    cleaned_df = pl.concat(cleaned_dfs)
    return cleaned_df

In [ ]:
def process_parquet_files(input_folder, static_df):
    # Liste des `encounterId` uniques du dataset statique
    encounter_ids = static_df["encounterId"].unique()

    # Création d'un dataframe de 0 à 180h 
    # 180h permet de prendre une marge de 12h supplémentaire afin de ne pas perdre de valeurs après suppression de l'offset
    intervalle_series = pl.Series("intervalle", range(0, 180))
    intervalle_df = pl.DataFrame({"intervalle": intervalle_series})  
      
    # Créer l'intervalle standardisé de 0 à 180 heures pour chaque patient
    standard_intervals = (
        pl.DataFrame({"encounterId": encounter_ids})
        .join(
            intervalle_df,
            how="cross"  # Produit cartésien
        )
    )

    # Contrôle du nombre de lignes du dataset des encounters
    print("Standard intervals shape:", standard_intervals.shape)

    # Initialiser un dataframe standardisé
    merged_df = standard_intervals

    # Parcourir chaque fichier .parquet et le fusionner avec l'intervalle standardisé
    for file in os.listdir(input_folder):
        if file.endswith(".parquet"):
            # Nom de la feature basée sur le nom du fichier
            feature_name = os.path.splitext(file)[0]

            # Charger le fichier .parquet
            df = pl.read_parquet(os.path.join(input_folder, file))

            # Joindre avec l'intervalle standardisé
            merged_df = (
                merged_df.join(
                    df,
                    on=["encounterId", "intervalle"],
                    how="left"
                )
                .rename({feature_name: feature_name})  # Renommer pour conserver le nom original
            )
    merged_df = merged_df.drop(['pep', 'fio2'])
    reset_intervalle_df = reset_first_value(merged_df)
    
    return reset_intervalle_df.filter(pl.col('intervalle') < 168)


In [ ]:
print(f"Expected temporal dataset lenght : {encounters.unique('encounterId').shape[0]*168}")

In [ ]:

temporal_week = process_parquet_files(OUTPUT_FOLDER, encounters)

6813180

In [ ]:
temporal_fold = f'/data2/poette.m/dypo/{VERSION}/3.analysis/times_series/{DATASET}/'
temporal_week.write_parquet(temporal_fold + 'one_week.parquet')

## Cleaning Report

In [ ]:
missing_encounters_folder = os.path.join(INPUT_FOLDER, 'missing_encounters')

In [ ]:

for filename in os.listdir(OUTPUT_FOLDER):
    print(f'--------{filename}----------')
    if filename.endswith(".parquet"):
        # Charger le dataset .parquet
        feature_data = pl.read_parquet(os.path.join(OUTPUT_FOLDER, filename))
        
        # Récupérer les encounterId du dataset .parquet
        feature_encounters = feature_data.select("encounterId").unique()
        
        # Trouver les lignes du dataset encounters n'apparaissant pas dans la feature
        missing_encounters = encounters.join(feature_encounters, on="encounterId", how="anti")
        
        # Afficher le résultat
        print(f"Encounters manquants: {missing_encounters.shape[0]}/{encounters.unique('encounterId').shape[0]}")
        missing_filename = os.path.join(missing_encounters_folder, filename.replace('cleared', 'missing'))
        #missing_encounters.write_parquet(missing_filename)